In [1]:
#pkg imports
import http.client
import json
import csv
import placekey
import pandas as pd
from placekey.api import PlacekeyAPI

In [106]:
#section for configuring input files

fileConfig1 = {
    "filepath": "propertySales",
    "parsetype": "str",
    "query_id": "sale_id",
    "street_address": "address",
    "city": "",
    #"region": "",
    "postal_code": "",
    "location_name": "",
    "latitude":"Y",
    "longitude":"X"
}
fileConfig2 = {
    "filepath": "blightData",
    "parsetype": "str",
    "query_id": "ticket_id",
    "street_address": "mailing_address",
    "city": "city",
    "region": "state",
    "postal_code": "zip_code",
    "location_name": "",
    "latitude":"Y",
    "longitude":"X"
}
apiParams = [
    "query_id",
    "latitude",
    "longitude",
    #"location_name",
    "street_address",
    "city",
    "postal_code",
    "region"
]
defaultParams1 = {
    "iso_country_code": "US",
    "region": "MI",
    "city": "Detroit"
}
defaultParams2 = {
    "iso_country_code": "US",
}
#output / api config
outputname = "output.csv"

apikey = "arYrLpPOiRUUHJfSzWUWmAGuKdnllvcO"
pk_api = PlacekeyAPI(apikey)


In [7]:
dataset1 = pd.read_csv(fileConfig1['filepath'], dtype= object)
dataset2 = pd.read_csv(fileConfig2['filepath'],  dtype= object)


In [99]:
#debug cell
print(dataset1.columns)
print(dataset2.columns)

Index(['X', 'Y', 'sale_id', 'parcel_number', 'address', 'street_number',
       'street_prefix', 'street_name', 'unit_number', 'sale_number',
       'sale_date', 'sale_price', 'grantor', 'grantee', 'liber_page',
       'term_of_sale', 'sale_verification', 'sale_instrument',
       'property_transferred_percentage', 'property_class_code',
       'economic_condition_factor_neigh', 'ESRI_OID'],
      dtype='object')
Index(['X', 'Y', 'ticket_id', 'ticket_number', 'agency_name', 'inspector_name',
       'violator_name', 'violation_street_number', 'violation_street_name',
       'violation_address', 'violation_zip_code', 'violator_id',
       'mailing_address_street_number', 'mailing_address_street_name',
       'mailing_address', 'city', 'state', 'zip_code', 'non_us_str_code',
       'country', 'violation_date', 'ticket_issued_time', 'hearing_date',
       'hearing_time', 'judgment_date', 'ordinance_law',
       'ordinance_description', 'disposition', 'fine_amount', 'admin_fee',
       'sta

In [104]:
#function for building api dataframe
def build_pk_df(df: pd.DataFrame, config: dict, defaults:dict):
    dfColumnMap = {}
    unusedParams = []
    for param in config.keys():
        if config[param] != "" and param in apiParams:
            dfColumnMap[config[param]] = param
        elif param in apiParams:
            unusedParams.append(param)
    api_df: pd.DataFrame = df.rename(columns=dfColumnMap)
    cols = list(dfColumnMap.values())
    api_df = api_df[cols]
    for uParam in unusedParams: 
        api_df[uParam] = None
    for p in defaults.keys():
        api_df[p] = defaults[p]
    api_df["latitude"] = pd.to_numeric(api_df["latitude"])
    api_df["longitude"] = pd.to_numeric(api_df["longitude"])
    return api_df

    

In [90]:
def getPlacekeys(api_df):
    api_json = json.loads(api_df.to_json(orient="records"))
    api_response = pk_api.lookup_placekeys(api_json)
    response_df = pd.read_json(json.dumps(api_response), dtype={'query_id': str})
    
    return response_df

In [91]:
def mergeData(placekey: pd.DataFrame, original: pd.DataFrame, query_id: str):
    merged_df = pd.merge(original, placekey, right_on= 'query_id', left_on=query_id, how='left')
    if 'error' in merged_df.columns:
        merged_df = merged_df.drop('error', axis=1)
    merged_df = merged_df.drop('query_id', axis=1)
    pk = merged_df.pop('placekey')
    merged_df.insert(0, 'placekey', pk)
    return merged_df

In [125]:
testData1 = dataset1.head(10000)
pk1_df = build_pk_df(testData1, fileConfig1, defaultParams1)
placekeys1 = getPlacekeys(pk1_df)

testData2 = dataset2.head(10000)
pk2_df = build_pk_df(testData2, fileConfig2, defaultParams2)
placekeys2 = getPlacekeys(pk2_df)

/var/folders/t2/nffj08cn41s_fmhrkwg75jq40000gn/T/ipykernel_22673/1732881984.py:4: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  response_df = pd.read_json(json.dumps(api_response), dtype={'query_id': str})
/var/folders/t2/nffj08cn41s_fmhrkwg75jq40000gn/T/ipykernel_22673/1732881984.py:4: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  response_df = pd.read_json(json.dumps(api_response), dtype={'query_id': str})


In [126]:
merged1: pd.DataFrame = mergeData(placekeys1, testData1, "sale_id")
merged2: pd.DataFrame = mergeData(placekeys2, testData2, "ticket_id")


In [121]:
print(merged1)
print(merged2)

                    placekey                  X                 Y  sale_id  \
0            226@63v-4jp-grk  -83.2244456079999  42.3881387340001  3749837   
1            22w@63v-456-h89      -83.210322565      42.438586871  4210008   
2            23d@63v-458-gx5      -83.209318776      42.415134218  4210338   
3            22r@63v-4jh-8sq      -83.161542697  42.3519161610001  3775822   
4            234@63v-48v-33q  -83.0740138319999  42.3469416950001  3663543   
...                      ...                ...               ...      ...   
4995         226@63v-44h-r8v  -83.0652287679999  42.4398406920001  4271219   
4996  0io2vdzlyk@63v-44v-59f      -83.115576097      42.361973339  4271220   
4997         23b@63v-486-m6k       -83.11501176      42.363990233  4271221   
4998         22p@63v-4hq-xt9        -83.1543794       42.38964975  4271222   
4999         22d@63v-4b3-26k  -82.9858325719999  42.4279330890001  4271223   

     parcel_number              address street_number street_pr

In [127]:
def mergeDataSets(data1, data2):
    df = data1.merge(data2, how='inner',on='placekey')
    return df
final = mergeDataSets(merged1, merged2)
print(final)
print(final.columns)
print(len(final))
final.to_csv(outputname, index=False)

            placekey                X_x               Y_x  sale_id  \
0    22d@63v-457-3kf      -83.225165621  42.4351922770001  4137441   
1    225@63v-4b3-2hq       -82.98520245  42.4285619500001  4347123   
2    23n@63v-4j4-3kf  -83.1819290989999  42.4108766370001  4186399   
3    227@63v-48z-kzz      -82.922157219  42.4020406700001  4251432   
4    23m@63v-455-tgk      -83.154194327      42.421802229  4661279   
..               ...                ...               ...      ...   
141  229@63v-4j6-9mk      -83.144583703       42.33553671  4042808   
142  229@63v-4j6-9mk      -83.144583703       42.33553671  4042808   
143  229@63v-4j6-9mk      -83.144583703       42.33553671  4042808   
144  224@63v-4dx-9vf  -83.2867776149999  42.4389900690001  3787528   
145  22t@63v-4g2-fxq      -83.202503087      42.338036023  4102727   

    parcel_number           address street_number street_prefix street_name  \
0       22080084.   19516 GREENVIEW         19516           NaN   GREENVIEW   
1